# Team : Chaos
## Idea : Face Recognition to walk-in into the office

#### Abstract : Walk in to the office, without the need of swiping your card. Automatic face recognition based entry and exit system.

#### Theme : Colleague Experience

#### Face recognition problems commonly fall into two categories: 

- **Face Verification** - "is this the claimed person?". For example, at some airports, you can pass through customs by letting a system scan your passport and then verifying that you (the person carrying the passport) are the correct person. A mobile phone that unlocks using your face is also using face verification. This is a 1:1 matching problem. 
- **Face Recognition** - "who is this person?". For example, the video below of some employees entering the office without needing to otherwise identify themselves. This is a 1:K matching problem. 


In [1]:
import numpy as np
import os
import cv2

from predictor import Predict
from register import Register

In [2]:
register = Register()

for img in os.listdir('known_faces'):
    name = os.path.splitext(img)[0]
    if name == '.ipynb_checkpoints':
        continue
    print("Known face : {}".format(name))
    register.register_user(name, os.path.join('known_faces', img))


Known face : vivek
Known face : pradeep
Known face : anish
Known face : asha
Known face : kirti
Known face : bharath
Known face : akshat
Known face : vishnu


In [3]:
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

video_capture = cv2.VideoCapture(0)

In [ ]:
while True:

    ret, frame = video_capture.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        face_locations, face_names = Predict.get_predictions(rgb_small_frame, register)

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        #cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, top - 20), font, 1.0, (0, 0, 255), 1)


    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

### References:

- Florian Schroff, Dmitry Kalenichenko, James Philbin (2015). [FaceNet: A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/pdf/1503.03832.pdf)
- Our implementation also took a lot of inspiration from the official FaceNet github repository: https://github.com/davidsandberg/facenet 
- Implementation of Facenet available as Pip Package (face-recognition) : https://pypi.org/project/face_recognition/


In [6]:
from IPython.display import HTML
HTML('<iframe src="resources/Motivation.mp4" width="700" height="394" frameborder="0" webkitallowfullscreen mozallowfullscreen allowfullscreen></iframe><p><a href="https://vimeo.com/26763844">BAXTER DURY - CLAIRE (Dir Cut)</a> from <a href="https://vimeo.com/dannysangra">Danny Sangra</a> on <a href="https://vimeo.com">Vimeo</a>.</p>')


#### Business Value : 
1. Travelling across offices would be easier. No need to raised request on IIQ and swim through tens of PRC groups for activating the access. 
2. Reduce walk-in time. Currently, in most of our facilites, colleagues first swipe their RFC card and then do the bio-metric. This process easily take about 5-7 seconds per day.

#### Technical Innovation
1. To best of our knoweldge, first solution which is build purely on Deep Learning.
2. Agnostic from deployment systems.